# RNN Composing Music

#Narratives

#Step1: Import Package and Function

Firstly, we need to import functions, which helps us to clean data and train models. And we also need packages, like tensorflow, for us to train our model.

In [1]:
import model_tb as model
import data

import os
import pickle
import sys
import tensorflow as tf
import argparse

ModuleNotFoundError: No module named 'model_tb'

We need to know wheteher this model is pre-trained or not. We will check for the pre-trained model, if not, the model will be trained by our model.biaxial_model function. 

In [ ]:
cache_name = 'cache.pkl'
model_name = None


if not os.path.exists(cache_name):
    composers = input("Enter composers separated by spaces: ").split()
    all_pieces = {}

    if len(composers)==0:
        all_pieces.update(data.getpices(path="midis", mode='all'))
    else:
        for c in composers:
            all_pieces.update(data.getpices(path="../midis", composer=c))

    cache = data.initialize_cache(all_pieces, save_loc=cache_name)
else:
    with open(cache_name, 'rb') as f:
        cache = pickle.load(f)

# Build and load the pre-existing model if it exists
print('Building model')
music_model = model.biaxial_model(t_layer_sizes=[300,300],
    n_layer_sizes=[100,50],
    trainer = tf.train.RMSPropOptimizer(0.005))

print('Start training')
music_model.train(cache, batch_size=5, max_epoch=10000,
    predict_freq=100, pre_trained_model=model_name)

# Predict

The pre-trained models name is biaxial_rnn_1524342232, which is trainned by 500 pop music. We can use this model to predict music and the generated music 

Although the training can be run in parallel, this is not possible when performing inference. To perform inference without teacher’s forcing, the network output at each timestep is used as input for the next timestep, requiring inference to be performed sequentially. We first get an initializer seed, sampled from the starting timestep of a measure in a random piece, as the last time step information. This input is fed through one timestep in the time network and then through the entire note dimension recurrently. The output is a complete “chord” distribution for one timestep. This output is then mapped to a new set of input vectors for each note, which is then processed through the biaxial network in a similar fashion. Once a sequence of chords with the desired length has been produced, the outputs are concatenated along the time dimension.

In [ ]:
model_name = "biaxial_rnn_1524342232"
    
    if not os.path.exists(cache_name):
        composers = input("Enter composers separated by spaces: ").split()
        all_pieces = {}
        
        if len(composers)==0:
            all_pieces.update(data.getpices(path="midis", mode='all'))
        else:
            for c in composers:
                all_pieces.update(data.getpices(path="../midis", composer=c))

        cache = data.initialize_cache(all_pieces, save_loc=cache_name)
    else:
        with open(cache_name, 'rb') as f:
            cache = pickle.load(f)
    
    # Building model
    print('Building model')
    music_model = model.biaxial_model(t_layer_sizes=[300,300], n_layer_sizes=[100,50], trainer=tf.train.RMSPropOptimizer(0.005))
    
    print('Start predicting')
    music_model.predict(cache,model_name,step=320,conservativity=1,n=20,saveto='predict songs')